In [ ]:
"""
Problem 1: Q-Learning on GridWorld - COMPLETE SOLUTION

This solution implements Q-learning to solve a 5x5 GridWorld environment.
"""

import numpy as np
import matplotlib.pyplot as plt

class GridWorld:
    """
    5x5 GridWorld Environment

    S . . . .
    . # . # .
    . . . # .
    . # . . .
    . . . . G

    S = Start (0,0)
    G = Goal (4,4)
    # = Wall (cannot pass)
    . = Empty cell
    """

    def __init__(self):
        self.grid_size = 5
        self.walls = [(1,1), (1,3), (2,3), (3,1)]
        self.start = (0, 0)
        self.goal = (4, 4)
        self.current_pos = self.start

        # Create mapping of valid positions to state indices
        self.valid_positions = []
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if (i, j) not in self.walls:
                    self.valid_positions.append((i, j))

        self.n_states = len(self.valid_positions)
        print(f"GridWorld initialized with {self.n_states} valid states")

    def reset(self):
        """Reset to start position. Return initial state."""
        self.current_pos = self.start
        return self.state_to_index(self.start)

    def step(self, action):
        """
        Take action and return (next_state, reward, done)
        Actions: 0=UP, 1=DOWN, 2=LEFT, 3=RIGHT
        """
        row, col = self.current_pos

        # Calculate new position based on action
        if action == 0:  # UP
            new_pos = (row - 1, col)
        elif action == 1:  # DOWN
            new_pos = (row + 1, col)
        elif action == 2:  # LEFT
            new_pos = (row, col - 1)
        else:  # RIGHT (action == 3)
            new_pos = (row, col + 1)

        # Check if new position is valid
        new_row, new_col = new_pos

        # Hit wall or boundary - stay in place, negative reward
        if (new_pos in self.walls or
            new_row < 0 or new_row >= self.grid_size or
            new_col < 0 or new_col >= self.grid_size):
            reward = -1
            new_pos = self.current_pos
        # Move to goal - positive reward
        elif new_pos == self.goal:
            reward = 10
            self.current_pos = new_pos
            return self.state_to_index(new_pos), reward, True
        # Normal move - small negative reward (encourages efficiency)
        else:
            reward = -0.1
            self.current_pos = new_pos

        next_state = self.state_to_index(self.current_pos)
        done = (self.current_pos == self.goal)

        return next_state, reward, done

    def state_to_index(self, state):
        """Convert (row, col) to unique state index"""
        try:
            return self.valid_positions.index(state)
        except ValueError:
            # Invalid state (wall)
            return -1

    def index_to_state(self, index):
        """Convert state index back to (row, col)"""
        if 0 <= index < len(self.valid_positions):
            return self.valid_positions[index]
        return None


def train_qlearning(env, episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.1):
    """
    Train Q-learning agent

    Args:
        env: GridWorld environment
        episodes: Number of training episodes
        alpha: Learning rate
        gamma: Discount factor
        epsilon: Exploration rate

    Returns:
        Q: Q-table (numpy array)
        rewards: List of total rewards per episode
    """
    n_states = env.n_states
    n_actions = 4

    # Initialize Q-table with zeros
    Q = np.zeros((n_states, n_actions))

    # Track rewards
    rewards_per_episode = []

    print(f"Training Q-learning for {episodes} episodes...")
    print(f"Parameters: alpha={alpha}, gamma={gamma}, epsilon={epsilon}")
    print("-" * 60)

    for episode in range(episodes):
        # Reset environment
        state = env.reset()
        total_reward = 0
        steps = 0
        max_steps = 100  # Prevent infinite loops

        while steps < max_steps:
            # Epsilon-greedy action selection
            if np.random.random() < epsilon:
                # Explore: random action
                action = np.random.randint(n_actions)
            else:
                # Exploit: best action from Q-table
                action = np.argmax(Q[state])

            # Take action
            next_state, reward, done = env.step(action)

            # Q-learning update rule:
            # Q(s,a) ← Q(s,a) + α[r + γ max Q(s',a') - Q(s,a)]
            best_next_action = np.argmax(Q[next_state])
            td_target = reward + gamma * Q[next_state, best_next_action]
            td_error = td_target - Q[state, action]
            Q[state, action] = Q[state, action] + alpha * td_error

            total_reward += reward
            state = next_state
            steps += 1

            if done:
                break

        rewards_per_episode.append(total_reward)

        # Print progress
        if (episode + 1) % 100 == 0:
            avg_reward = np.mean(rewards_per_episode[-100:])
            print(f"Episode {episode + 1}: Avg Reward (100) = {avg_reward:.2f}")

    print("-" * 60)
    print(f"Training complete!")
    print(f"Final average reward (last 100): {np.mean(rewards_per_episode[-100:]):.2f}")

    return Q, rewards_per_episode


def visualize_policy(env, Q):
    """Visualize learned policy as arrows"""
    action_symbols = ['↑', '↓', '←', '→']

    print("\nLearned Policy:")
    print("-" * 25)

    for i in range(env.grid_size):
        row_display = []
        for j in range(env.grid_size):
            if (i, j) == env.start:
                row_display.append('S')
            elif (i, j) == env.goal:
                row_display.append('G')
            elif (i, j) in env.walls:
                row_display.append('#')
            else:
                state_idx = env.state_to_index((i, j))
                if state_idx >= 0:
                    best_action = np.argmax(Q[state_idx])
                    row_display.append(action_symbols[best_action])
                else:
                    row_display.append('?')

        print('  '.join(row_display))

    print("-" * 25)


def plot_results(rewards):
    """Plot training rewards"""
    plt.figure(figsize=(12, 5))

    # Plot 1: Raw rewards
    plt.subplot(1, 2, 1)
    plt.plot(rewards, alpha=0.6)
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('Training Rewards per Episode')
    plt.grid(True, alpha=0.3)

    # Plot 2: Moving average
    plt.subplot(1, 2, 2)
    window = 50
    if len(rewards) >= window:
        moving_avg = [np.mean(rewards[max(0, i-window+1):i+1])
                     for i in range(len(rewards))]
        plt.plot(moving_avg, linewidth=2)
        plt.xlabel('Episode')
        plt.ylabel('Average Reward')
        plt.title(f'Moving Average ({window} episodes)')
        plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('problem1_results.png', dpi=300, bbox_inches='tight')
    print("\nPlot saved as 'problem1_results.png'")
    plt.close()


def test_agent(env, Q, num_tests=10):
    """Test the trained agent"""
    print(f"\nTesting agent for {num_tests} episodes...")

    successes = 0
    total_steps = []

    for test in range(num_tests):
        state = env.reset()
        steps = 0
        max_steps = 100

        while steps < max_steps:
            # Use greedy policy (no exploration)
            action = np.argmax(Q[state])
            state, reward, done = env.step(action)
            steps += 1

            if done:
                successes += 1
                total_steps.append(steps)
                break

    print(f"Success rate: {successes}/{num_tests} ({100*successes/num_tests:.1f}%)")
    if total_steps:
        print(f"Average steps to goal: {np.mean(total_steps):.1f}")


# Main execution
if __name__ == "__main__":
    print("="*60)
    print("PROBLEM 1: Q-LEARNING ON GRIDWORLD")
    print("="*60)

    # Create environment
    env = GridWorld()

    # Train agent
    Q, rewards = train_qlearning(
        env,
        episodes=1000,
        alpha=0.1,
        gamma=0.99,
        epsilon=0.1
    )

    # Visualize policy
    visualize_policy(env, Q)

    # Plot results
    plot_results(rewards)

    # Test agent
    test_agent(env, Q, num_tests=20)

    # Print Q-table statistics
    print(f"\nQ-table Statistics:")
    print(f"  Shape: {Q.shape}")
    print(f"  Non-zero values: {np.count_nonzero(Q)}/{Q.size}")
    print(f"  Max Q-value: {np.max(Q):.2f}")
    print(f"  Min Q-value: {np.min(Q):.2f}")

    print("\n" + "="*60)
    print("PROBLEM 1 COMPLETE!")
    print("="*60)

In [ ]:
"""
Problem 2: Deep Q-Network Completion - COMPLETE SOLUTION

This solution completes the DQN implementation for CartPole-v1
"""

import torch
import torch.nn as nn
import torch.optim as optim
import gym
import numpy as np
from collections import deque
import random
import matplotlib.pyplot as plt

class DQN(nn.Module):
    """Deep Q-Network architecture"""

    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()

        # Neural network architecture
        # Input: state_dim -> Hidden: 128 -> Hidden: 128 -> Output: action_dim
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        """Forward pass through the network"""
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class ReplayBuffer:
    """Experience replay buffer"""

    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        """Sample random mini-batch"""
        # Randomly sample batch_size transitions
        batch = random.sample(self.buffer, batch_size)

        # Unpack and convert to numpy arrays
        states, actions, rewards, next_states, dones = zip(*batch)

        return (
            np.array(states, dtype=np.float32),
            np.array(actions, dtype=np.int64),
            np.array(rewards, dtype=np.float32),
            np.array(next_states, dtype=np.float32),
            np.array(dones, dtype=np.float32)
        )

    def __len__(self):
        return len(self.buffer)


class DQNAgent:
    """DQN Agent"""

    def __init__(self, state_dim, action_dim):
        self.action_dim = action_dim
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

        # Initialize Q-network and target network
        self.q_network = DQN(state_dim, action_dim)
        self.target_network = DQN(state_dim, action_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()

        # Initialize optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=0.001)

        # Initialize replay buffer
        self.replay_buffer = ReplayBuffer(capacity=10000)

        self.batch_size = 64

    def select_action(self, state, training=True):
        """Epsilon-greedy action selection"""
        if training and np.random.random() < self.epsilon:
            # Explore: random action
            return np.random.randint(self.action_dim)
        else:
            # Exploit: best action from Q-network
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            with torch.no_grad():
                q_values = self.q_network(state_tensor)
            return q_values.argmax().item()

    def train_step(self, batch_size=64):
        """Training step using experience replay"""
        if len(self.replay_buffer) < batch_size:
            return 0

        # Sample batch from replay buffer
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(batch_size)

        # Convert to PyTorch tensors
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones)

        # Compute current Q-values
        # Q(s,a) for the actions that were taken
        current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1))

        # Compute target Q-values using target network
        # Target: r + γ * max_a' Q̂(s', a')
        with torch.no_grad():
            next_q_values = self.target_network(next_states).max(1)[0]
            target_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        # Compute loss (MSE between current and target Q-values)
        loss = nn.MSELoss()(current_q_values.squeeze(), target_q_values)

        # Backpropagate and update network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        return loss.item()

    def update_target_network(self):
        """Copy weights from Q-network to target network"""
        self.target_network.load_state_dict(self.q_network.state_dict())


def train_cartpole(episodes=500):
    """Train DQN on CartPole-v1"""

    print("="*60)
    print("PROBLEM 2: DQN ON CARTPOLE-V1")
    print("="*60)

    # Create environment
    env = gym.make('CartPole-v1')
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    print(f"Environment: CartPole-v1")
    print(f"State dimension: {state_dim}")
    print(f"Action dimension: {action_dim}")
    print("-" * 60)

    # Create agent
    agent = DQNAgent(state_dim, action_dim)

    # Training metrics
    rewards = []
    losses = []

    for episode in range(episodes):
        # Reset environment
        result = env.reset()
        if isinstance(result, tuple):
            state = result[0]
        else:
            state = result

        total_reward = 0
        episode_losses = []
        steps = 0
        done = False

        while not done:
            # Select action
            action = agent.select_action(state, training=True)

            # Take step in environment
            result = env.step(action)
            if len(result) == 5:
                next_state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                next_state, reward, done, _ = result

            # Store transition in replay buffer
            agent.replay_buffer.push(state, action, reward, next_state, done)

            # Train agent
            loss = agent.train_step()
            if loss > 0:
                episode_losses.append(loss)

            # Update state and reward
            state = next_state
            total_reward += reward
            steps += 1

        # Update target network every 10 episodes
        if (episode + 1) % 10 == 0:
            agent.update_target_network()

        # Track rewards
        rewards.append(total_reward)
        if episode_losses:
            losses.append(np.mean(episode_losses))

        # Print progress
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(rewards[-100:]) if len(rewards) >= 100 else np.mean(rewards)
            print(f"Episode {episode + 1}/{episodes}")
            print(f"  Reward: {total_reward:.1f} | Avg (100): {avg_reward:.1f}")
            print(f"  Steps: {steps} | Epsilon: {agent.epsilon:.3f}")
            if losses:
                print(f"  Loss: {losses[-1]:.4f}")
            print("-" * 60)

        # Check if solved
        if len(rewards) >= 100 and np.mean(rewards[-100:]) >= 475:
            print(f"\n🎉 CartPole solved in {episode + 1} episodes!")
            print(f"Average reward (100 episodes): {np.mean(rewards[-100:]):.2f}")
            break

    env.close()

    # Plot results
    plot_results(rewards, losses)

    return agent, rewards


def plot_results(rewards, losses):
    """Plot training results"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Plot rewards
    axes[0].plot(rewards, alpha=0.6, label='Episode Reward')
    if len(rewards) >= 100:
        moving_avg = [np.mean(rewards[max(0, i-99):i+1]) for i in range(len(rewards))]
        axes[0].plot(moving_avg, linewidth=2, label='Moving Avg (100)')
    axes[0].axhline(y=475, color='r', linestyle='--', label='Solved Threshold')
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Reward')
    axes[0].set_title('Training Rewards')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Plot losses
    if losses:
        axes[1].plot(losses, alpha=0.6, label='Loss')
        if len(losses) >= 50:
            moving_avg = [np.mean(losses[max(0, i-49):i+1]) for i in range(len(losses))]
            axes[1].plot(moving_avg, linewidth=2, label='Moving Avg (50)')
        axes[1].set_xlabel('Episode')
        axes[1].set_ylabel('Loss')
        axes[1].set_title('Training Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('problem2_results.png', dpi=300, bbox_inches='tight')
    print("\nPlot saved as 'problem2_results.png'")
    plt.close()


def test_agent(agent, episodes=10):
    """Test trained agent"""
    env = gym.make('CartPole-v1')

    print(f"\nTesting agent for {episodes} episodes...")
    test_rewards = []

    for episode in range(episodes):
        result = env.reset()
        if isinstance(result, tuple):
            state = result[0]
        else:
            state = result

        total_reward = 0
        done = False

        while not done:
            action = agent.select_action(state, training=False)
            result = env.step(action)

            if len(result) == 5:
                state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                state, reward, done, _ = result

            total_reward += reward

        test_rewards.append(total_reward)
        print(f"Test Episode {episode + 1}: Reward = {total_reward:.1f}")

    env.close()

    print(f"\nTest Results:")
    print(f"  Average: {np.mean(test_rewards):.2f}")
    print(f"  Std Dev: {np.std(test_rewards):.2f}")
    print(f"  Min/Max: {np.min(test_rewards):.1f} / {np.max(test_rewards):.1f}")


# Main execution
if __name__ == "__main__":
    # Train agent
    agent, rewards = train_cartpole(episodes=500)

    # Test agent
    test_agent(agent, episodes=10)

    # Final statistics
    print("\n" + "="*60)
    print("TRAINING STATISTICS")
    print("="*60)
    print(f"Total episodes: {len(rewards)}")
    print(f"Final avg reward (100): {np.mean(rewards[-100:]):.2f}")
    print(f"Best episode reward: {np.max(rewards):.1f}")
    print(f"Solved: {'Yes' if np.mean(rewards[-100:]) >= 475 else 'No'}")
    print("="*60)
    print("PROBLEM 2 COMPLETE!")
    print("="*60)

In [ ]:
"""
Problem 3: Analysis and Debugging - COMPLETE SOLUTION

This solution identifies and fixes bugs in the FrozenLake Q-learning implementation
"""

import gym
import numpy as np
import matplotlib.pyplot as plt

# ============================================================================
# BUG ANALYSIS
# ============================================================================

"""
BUG 1: Q-table initialization
Location: Line 15
Description: Q-table is initialized but might have dimension issues
Impact: Incorrect Q-table shape could cause indexing errors
Fix: Ensure proper dimensions match state and action spaces

BUG 2: Action selection (Line 33)
Location: Line 33 (np.argmin instead of np.argmax)
Description: Using argmin (minimum Q-value) instead of argmax (maximum Q-value)
Impact: Agent chooses WORST actions instead of BEST actions - completely wrong!
Fix: Change np.argmin(Q[state]) to np.argmax(Q[state])

BUG 3: Q-learning update rule (Line 40)
Location: Line 40
Description: Missing learning rate (alpha) in the update rule
             Current: Q[state, action] = reward + gamma * np.max(Q[next_state])
             Correct: Q[state, action] += alpha * (target - current_q)
Impact: Overwrites Q-values instead of gradually updating them
Fix: Use proper temporal difference update with learning rate

BUG 4: Epsilon decay (Line 51)
Location: No epsilon decay implemented
Description: Epsilon stays constant at 0.3, never decreases
Impact: Agent never shifts from exploration to exploitation
Fix: Add epsilon decay: epsilon = max(epsilon_min, epsilon * decay_rate)
"""

# ============================================================================
# BUGGY CODE (For Reference)
# ============================================================================

def buggy_train_frozenlake(episodes=10000):
    """
    BUGGY VERSION - For demonstration purposes
    """
    env = gym.make('FrozenLake-v1', is_slippery=True)

    n_states = env.observation_space.n
    n_actions = env.action_space.n

    # Bug 1: Q-table initialization (this is actually OK, but could note dimensions)
    Q = np.zeros((n_states, n_actions))

    alpha = 0.8  # Learning rate
    gamma = 0.95  # Discount factor
    epsilon = 0.3  # Exploration rate

    rewards = []

    for episode in range(episodes):
        result = env.reset()
        state = result[0] if isinstance(result, tuple) else result
        total_reward = 0
        done = False

        while not done:
            # Bug 2: Action selection issue - using argmin instead of argmax!
            if np.random.random() < epsilon:
                action = np.random.randint(n_actions)
            else:
                action = np.argmin(Q[state])  # BUG: Should be argmax!

            result = env.step(action)
            if len(result) == 5:
                next_state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                next_state, reward, done, _ = result

            # Bug 3: Update rule issue - missing learning rate and current Q-value
            Q[state, action] = reward + gamma * np.max(Q[next_state])  # BUG: Wrong update!

            state = next_state
            total_reward += reward

        rewards.append(total_reward)

        # Bug 4: Epsilon handling - no epsilon decay!

    success_rate = np.mean(rewards[-100:])
    print(f"Buggy version success rate: {success_rate}")

    return Q, rewards


# ============================================================================
# FIXED CODE
# ============================================================================

def fixed_train_frozenlake(episodes=10000):
    """
    FIXED VERSION - All bugs corrected
    """
    env = gym.make('FrozenLake-v1', is_slippery=True)

    n_states = env.observation_space.n
    n_actions = env.action_space.n

    print(f"FrozenLake Environment:")
    print(f"  States: {n_states}")
    print(f"  Actions: {n_actions}")
    print("-" * 60)

    # FIX 1: Q-table initialization (this was OK, but adding comment)
    Q = np.zeros((n_states, n_actions))  # Properly shaped Q-table

    # Hyperparameters
    alpha = 0.1  # Learning rate (lowered from 0.8 for more stable learning)
    gamma = 0.99  # Discount factor (increased for better long-term planning)
    epsilon = 1.0  # Initial exploration rate (start high)
    epsilon_min = 0.01  # Minimum exploration rate
    epsilon_decay = 0.9995  # Epsilon decay rate

    rewards = []

    print(f"Training with:")
    print(f"  Learning rate (alpha): {alpha}")
    print(f"  Discount factor (gamma): {gamma}")
    print(f"  Initial epsilon: {epsilon}")
    print(f"  Epsilon decay: {epsilon_decay}")
    print("-" * 60)

    for episode in range(episodes):
        result = env.reset()
        state = result[0] if isinstance(result, tuple) else result
        total_reward = 0
        done = False
        steps = 0
        max_steps = 100  # Prevent infinite loops

        while not done and steps < max_steps:
            # FIX 2: Action selection - use argmax for exploitation!
            if np.random.random() < epsilon:
                action = np.random.randint(n_actions)  # Explore
            else:
                action = np.argmax(Q[state])  # Exploit (FIXED: was argmin)

            result = env.step(action)
            if len(result) == 5:
                next_state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                next_state, reward, done, _ = result

            # FIX 3: Proper Q-learning update rule with learning rate
            # Q(s,a) ← Q(s,a) + α[r + γ max Q(s',a') - Q(s,a)]
            current_q = Q[state, action]
            max_next_q = np.max(Q[next_state])
            target_q = reward + gamma * max_next_q
            Q[state, action] = current_q + alpha * (target_q - current_q)

            state = next_state
            total_reward += reward
            steps += 1

        rewards.append(total_reward)

        # FIX 4: Epsilon decay - gradually reduce exploration
        epsilon = max(epsilon_min, epsilon * epsilon_decay)

        # Print progress
        if (episode + 1) % 1000 == 0:
            success_rate = np.mean(rewards[-100:])
            print(f"Episode {episode + 1}: Success rate (100) = {success_rate:.3f}, Epsilon = {epsilon:.3f}")

    env.close()

    success_rate = np.mean(rewards[-100:])
    print(f"\nFinal success rate: {success_rate:.3f}")

    return Q, rewards


def compare_buggy_vs_fixed():
    """
    Compare buggy and fixed implementations
    """
    print("="*60)
    print("PROBLEM 3: DEBUGGING Q-LEARNING")
    print("="*60)
    print("\nBUG ANALYSIS:")
    print("-" * 60)
    print("Bug 1: Q-table initialization")
    print("  Issue: Actually OK, but should verify dimensions")
    print("  Impact: Low")
    print("  Fix: Ensure Q = np.zeros((n_states, n_actions))")
    print()
    print("Bug 2: Action selection (CRITICAL)")
    print("  Issue: Using np.argmin instead of np.argmax")
    print("  Impact: Agent chooses WORST actions, not BEST")
    print("  Fix: Change to np.argmax(Q[state])")
    print()
    print("Bug 3: Q-learning update rule (CRITICAL)")
    print("  Issue: Missing learning rate, overwrites Q-values")
    print("  Impact: No gradual learning, unstable updates")
    print("  Fix: Q[s,a] += alpha * (target - Q[s,a])")
    print()
    print("Bug 4: Epsilon decay (IMPORTANT)")
    print("  Issue: Epsilon never decreases")
    print("  Impact: Agent never exploits learned knowledge")
    print("  Fix: epsilon = max(epsilon_min, epsilon * decay)")
    print("-" * 60)

    episodes = 5000

    # Train buggy version
    print("\n" + "="*60)
    print("Training BUGGY version...")
    print("="*60)
    Q_buggy, rewards_buggy = buggy_train_frozenlake(episodes=episodes)

    # Train fixed version
    print("\n" + "="*60)
    print("Training FIXED version...")
    print("="*60)
    Q_fixed, rewards_fixed = fixed_train_frozenlake(episodes=episodes)

    # Compare results
    print("\n" + "="*60)
    print("COMPARISON RESULTS")
    print("="*60)
    print(f"\nBuggy Implementation:")
    print(f"  Success rate: {np.mean(rewards_buggy[-100:]):.3f}")
    print(f"  Total successes: {sum(rewards_buggy)}")
    print(f"\nFixed Implementation:")
    print(f"  Success rate: {np.mean(rewards_fixed[-100:]):.3f}")
    print(f"  Total successes: {sum(rewards_fixed)}")
    print(f"\nImprovement: {(np.mean(rewards_fixed[-100:]) - np.mean(rewards_buggy[-100:])):.3f}")

    # Plot comparison
    plot_comparison(rewards_buggy, rewards_fixed)

    print("\n" + "="*60)
    print("PROBLEM 3 COMPLETE!")
    print("="*60)


def plot_comparison(rewards_buggy, rewards_fixed):
    """Plot comparison of buggy vs fixed implementation"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    window = 100

    # Moving averages
    ma_buggy = [np.mean(rewards_buggy[max(0, i-window+1):i+1])
                for i in range(len(rewards_buggy))]
    ma_fixed = [np.mean(rewards_fixed[max(0, i-window+1):i+1])
                for i in range(len(rewards_fixed))]

    # Plot 1: Success rates over time
    axes[0].plot(ma_buggy, label='Buggy Version', alpha=0.7, linewidth=2)
    axes[0].plot(ma_fixed, label='Fixed Version', alpha=0.7, linewidth=2)
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Success Rate (100-episode window)')
    axes[0].set_title('Learning Progress Comparison')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Plot 2: Final performance comparison
    final_buggy = np.mean(rewards_buggy[-100:])
    final_fixed = np.mean(rewards_fixed[-100:])

    axes[1].bar(['Buggy', 'Fixed'], [final_buggy, final_fixed],
                color=['red', 'green'], alpha=0.7)
    axes[1].set_ylabel('Success Rate')
    axes[1].set_title('Final Performance (last 100 episodes)')
    axes[1].set_ylim([0, 1])
    axes[1].grid(True, alpha=0.3, axis='y')

    # Add value labels on bars
    axes[1].text(0, final_buggy + 0.02, f'{final_buggy:.3f}',
                ha='center', fontweight='bold')
    axes[1].text(1, final_fixed + 0.02, f'{final_fixed:.3f}',
                ha='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig('problem3_comparison.png', dpi=300, bbox_inches='tight')
    print("\nComparison plot saved as 'problem3_comparison.png'")
    plt.close()


# Main execution
if __name__ == "__main__":
    compare_buggy_vs_fixed()